In [19]:
import pandas as pd

import timeit
import logging

# API Handling 
from dotenv import load_dotenv
import requests
import os
import json

In [20]:
# Paths
ruta_carpeta_establecimientos = "../../input_bauti/coordenadas_campos/"

# Path output
path_output = '../../input_bauti/coordenadas_campos/establecimientos_geolocalizados/'
name_output = 'establecimientos_geolocalizados_mapquest.xlsx'

#Logger path
path_logger = '../../logging.log'

In [21]:
def configure ():
    load_dotenv()
env = configure()

In [22]:
start = timeit.default_timer()
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter(fmt ='%(asctime)s - %(levelname)s - %(message)s', datefmt='%d-%m-%Y %H:%M')
handler = logging.FileHandler(path_logger)
handler.setLevel(logging.DEBUG)
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.info("----Inicio Geolocalizacion de establecimientos----")

In [23]:
hay_archivos_establecimientos_nuevo = (len(os.listdir(ruta_carpeta_establecimientos)) > 0)
archivo_establecimientos_nuevo = ""

if not hay_archivos_establecimientos_nuevo:
    logger.info("No hay nuevo archivo de establecimientos. Utilizando los establecimientos existentes")
    stop = timeit.default_timer()
    logger.info(f"Fin de ejecucion{stop-start}")
    exit(0)
    
else:
    archivo_establecimientos_nuevo = ruta_carpeta_establecimientos+os.listdir(ruta_carpeta_establecimientos)[0]
    customer_key = os.getenv('CUSTOMER_KEY')
    api_route = 'https://www.mapquestapi.com/directions/v2/route?key=%s' %(customer_key)
    api_destination = 'https://www.mapquestapi.com/geocoding/v1/reverse?key=%s' %(customer_key)

In [24]:
#carga de datos
try:
    df_cordenadas = pd.read_excel(archivo_establecimientos_nuevo)
    
    # Dropeo posibles columnas con nombres Unnamed
    cols_drop = df_cordenadas.columns[df_cordenadas.columns.str.contains('Unnamed')]
    if len(cols_drop) != 0:
        df_cordenadas.drop(columns=cols_drop, inplace=True)
        
except Exception as ex:
    logger.warning("Error en la carga de datos",exc_info=True)

In [25]:
# Aplicar la función lambda para concatenar latitud y longitud y crear una nueva columna llamada 'coordenadas'
df_cordenadas['coordenadas'] = df_cordenadas.apply(lambda row: f"{row['Lat']},{row['Long']}", axis=1)

In [26]:
# Obtengo las coordenadas de la planta de Bayer, y las convierto en json para pasarsela a requests.
bayer_lat = -34.2031478
bayer_lon = -60.6939654

def obtener_distancia_bayer(ubicacion, modo="truck"):
    """
        Obtiene la distancia y el tiempo entre dos ubicaciones utilizando MapQuest API.
        
        Parámetros:
            ubicacion (str): Coordenadas (latitud, longitud) de la ubicación.
            modo (str): Modo de transporte ('driving', 'walking', 'biking', 'truck' o 'transit'). Por defecto es 'truck'.

        Retorna:
            dict: Un diccionario con datos de la ruta.
        """
    try:
        parametros = {
            'from': f'{bayer_lat},{bayer_lon}',
            'to': ubicacion,
            'outFormat': 'json',
            'routeType': modo,
            'doReverseGeocode':'false',
            'unit' : 'k',
            'locale' : 'es_ES'
        }
        response = requests.get(api_route, params=parametros)

        datos = response.json()
        ruta = datos['route']

        distancia = ruta['distance'] # expresado en km
        distancia_value = ruta['distance']* 1000 # expresado en km

        time = ruta['time'] # expresado en segundos.
        realtime = ruta['realTime'] # expresado en segundos.
        # Divido los valres por 60 para obtener los minutos, ya que el output esta expresado en segundos.
        if realtime > time:
            duracion_min = realtime / 60
        else:
            duracion_min = time / 60

        # Obtengo la ciudad y el estado. Para esto genero una request nueva.
        # Parametros para el request.
        parametros = {
            'location' : ubicacion,
            'includeNearestIntersection' : 'true'
        }

        response = requests.get(api_destination, params=parametros)

        datos = response.json()

        # Obtengo la ciudad
        ciudad = datos['results'][0]['locations'][0]['adminArea5']

        # Obtengo el estado.
        estado = datos['results'][0]['locations'][0]['adminArea3']

    except:
        distancia, duracion_min,  ciudad, estado = 0, 0, 0, 0
        logger.warning(f"Error no se encuentra localizacion {ubicacion}")

    return distancia, distancia_value, duracion_min, ciudad, estado

In [28]:
df_cordenadas['distancia'], df_cordenadas['distancia_value'], df_cordenadas['duracion_min'], \
df_cordenadas['city'], df_cordenadas['state'] \
= zip(*df_cordenadas.apply(lambda row: obtener_distancia_bayer(row['coordenadas']), axis=1))

In [29]:
def min_a_hs (min):
    horas = int(min /60)
    minutos_restantes = int(min % 60)
    
    return '%s h %s min' %(horas, minutos_restantes)

In [30]:
df_cordenadas['duracion'] = df_cordenadas['duracion_min'].apply(min_a_hs)
df_cordenadas.rename(columns={
    'Zona' : 'zone', 
    'Campo' : 'establecimiento',
    'Good Supplier' : 'good_supplier',
    'Area' : 'area', 
    'Lat' : 'latitud', 
    'Long' : 'longitud',
    'distancia_value' :'distancia_value_mts'
    }, 
    inplace=True
)

# Cambiamos el orden de las columnas
column_order = [
    'zone', 
    'establecimiento',
    'good_supplier',
    'area', 
    'latitud', 
    'longitud', 
    'distancia',
    'distancia_value_mts', 
    'duracion', 
    'duracion_min' ,
    'city', 
    'state'
]
df_cordenadas = df_cordenadas[column_order]
df_cordenadas['distancia_value_mts'] = df_cordenadas['distancia_value_mts'].astype(int)

C:\Users\TALIGENT\AppData\Local\Temp\ipykernel_19348\3083860423.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cordenadas['distancia_value_mts'] = df_cordenadas['distancia_value_mts'].astype(int)


In [31]:
# Mapeamos las columnas de las provicias, para su correcta interpretacion.
provincias_argentinas = {
    'TM': 'Tucumán',
    'CT': 'Catamarca',
    'BA': 'Buenos Aires',
    'MZ': 'Mendoza',
    'SJ': 'San Juan',
    'SL': 'San Luis',
    'LR': 'La Rioja',
    'CB': 'Córdoba',
    'SE': 'Santiago del Estero',
    'CH': 'Chaco',
    'FO': 'Formosa',
    'ER': 'Entre Ríos',
    'MN': 'Misiones',
    'CO': 'Corrientes',
    'CN': 'Chubut',
    'RN': 'Río Negro',
    'NE': 'Neuquén',
    'SC': 'Santa Cruz',
    'TF': 'Tierra del Fuego, Antártida e Islas del Atlántico Sur',
    'SF': 'Santa Fe',
    'BA': 'Buenos Aires',
    'CABA': 'Ciudad Autónoma de Buenos Aires',
    'LP': 'La Pampa',
    'SG': 'Santiago del Estero',
}

df_cordenadas['state'] = df_cordenadas['state'].map(provincias_argentinas)

C:\Users\TALIGENT\AppData\Local\Temp\ipykernel_19348\3864384283.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cordenadas['state'] = df_cordenadas['state'].map(provincias_argentinas)


In [32]:
# Creo una nueva carpeta para este archivo y guardo el archivo.
if not os.path.exists(path_output):
    os.makedirs(path_output)

df_cordenadas.to_excel(path_output+name_output, index=False)